# Model input

In [ ]:
import cobra
from cobra.io import read_sbml_model
model = read_sbml_model('iJO1366.xml')

# FESOF
target: the design target(reaction id)

In [ ]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF
fseof = FSEOF(model)
result = fseof.run(target=model.reactions.EX_lys_L_LPAREN_e_RPAREN_)
df = result.data_frame
df.to_excel('lys_fesof.xlsx')

# OptKnock
max_knockouts: max number of knockouts allowed           
max_results: max number of different designs to return if found          
target: the design target(reaction id)              
biomass: the biomass id in the model

In [ ]:
from cameo.strain_design import OptKnock
optknock = OptKnock(model,fraction_of_optimum=0.1)
result = optknock.run(max_knockouts=5, target="EX_trp__L_e",biomass="BIOMASS_Ec_iJO1366_core_53p95M",max_result=5)
df = result.data_frame
df.to_excel('trp_optknock.xlsx')

# OptGene
target: the design target(reaction id)        
biomass: the biomass id in the model             
substrate: the main carbon source in the model
simulation_method: Any method from cameo.flux_analysis.simulation or equivalent ( fba(default), pfba, moma, lmoma, room )     
max_knockouts: max number of knockouts allowed            
max_evaluations: number of evaluations before stop( default:20000 )            
population_size: number of individuals in each generation( default:200 )           
growth_coupled: If true will use the minimum flux rate to compute the fitness

In [ ]:
from cameo.strain_design import OptGene
optgene = OptGene(model)
result = optgene.run(target=model.reactions.EX_trp__L_e,
                    biomass=model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                    substrate=model.metabolites.glc__D_e,max_knockouts=5,
                    simulation_method=fba,
                    max_evaluations=20000,population_size=200,
                    plot=False, growth_coupled=True)
df = result.data_frame
df.to_excel('trp_optgene.xlsx')

# OptSwap

In [ ]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
model.objective = model.reactions.EX_thr__L_e
model.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = 0.1
py = product_yield(model.reactions.EX_thr__L_e, model.reactions.EX_glc__D_e)
optswap = CofactorSwapOptimization(model=model, objective_function=py)
optswap.run(max_evaluations=2000, max_size=2)